In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the dataset
file_path = 'train.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

In [3]:
# Selecting key features for the model
key_features = ['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high']
data = data[key_features + ['price_doc']]

In [4]:
# Applying logarithmic transformations
data['full_sq_log'] = np.log1p(data['full_sq'])
data['leisure_count_500_log'] = np.log1p(data['leisure_count_500'])

In [5]:
# Handling any potential infinities or NaNs
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(0, inplace=True)

In [6]:
# Splitting the data into features (X) and target variable (y)
X = data.drop('price_doc', axis=1)
y = data['price_doc']

In [7]:
# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Feature Scaling using Robust Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [14]:
# Define the learning rate
learning_rate = 0.01  # You can change this value as needed

In [10]:
batch_size = 32 # Adjust the batch size

In [15]:
# Neural Network Design
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),  # Increased dropout rate
    Dense(32, activation='relu'),
    Dropout(0.2),  # Added dropout layer
    Dense(1, activation='linear')  # Output layer for regression
])


In [16]:
# Compiling the Model with a custom learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=[RootMeanSquaredError(name='rmse')])


In [ ]:
# Implementing Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [17]:
# Training the Model with the specified batch size
history = model.fit(X_train_scaled, y_train, batch_size=32, epochs=150, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping])

Epoch 1/150
4538/4538 [==============================] - 5s 933us/step - loss: 223194300547072.0000 - rmse: 14939689.0000 - val_loss: 196073528229888.0000 - val_rmse: 14002626.0000
Epoch 2/150
4538/4538 [==============================] - 4s 844us/step - loss: 195823514157056.0000 - rmse: 13993696.0000 - val_loss: 194671808610304.0000 - val_rmse: 13952484.0000
Epoch 3/150
4538/4538 [==============================] - 4s 875us/step - loss: 193540923588608.0000 - rmse: 13911899.0000 - val_loss: 191394866003968.0000 - val_rmse: 13834553.0000
Epoch 4/150
4538/4538 [==============================] - 4s 854us/step - loss: 191489053294592.0000 - rmse: 13837957.0000 - val_loss: 188410132168704.0000 - val_rmse: 13726257.0000
Epoch 5/150
4538/4538 [==============================] - 4s 852us/step - loss: 189546000023552.0000 - rmse: 13767571.0000 - val_loss: 185703614906368.0000 - val_rmse: 13627311.0000
Epoch 6/150
4538/4538 [==============================] - 4s 881us/step - loss: 186735967338496.

In [ ]:
# Model Summary
model.summary()

In [18]:
# Evaluating the model
results = model.evaluate(X_val_scaled, y_val)
print(f"Validation RMSE: {results[1]}")

1135/1135 [==============================] - 1s 589us/step - loss: 174759165820928.0000 - rmse: 13219651.0000
Validation RMSE: 13219651.0


In [ ]:
# Load the test dataset
test_file_path = 'test.csv'  # Replace with your test CSV file path
test_data = pd.read_csv(test_file_path)

# Apply the same preprocessing steps to the test dataset
test_data['full_sq_log'] = np.log1p(test_data['full_sq'])
test_data['leisure_count_500_log'] = np.log1p(test_data['leisure_count_500'])

# Handle potential infinities or NaNs
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.fillna(0, inplace=True)

# Select the same features used for training (including original and transformed features)
X_test = test_data[['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high', 'full_sq_log', 'leisure_count_500_log']]

# Scale the test data using the same scaler as the training data
X_test_scaled = scaler.transform(X_test)

# Make predictions
predictions = model.predict(X_test_scaled)

# Save predictions to a CSV file
submission = pd.DataFrame({
    'row ID': test_data['row ID'],  # Replace 'row ID' with the actual identifier column of your test dataset
    'price_doc': predictions.flatten()
})
submission.to_csv('neural-network.csv', index=False)

